## BT-LAB .mpt with loops to csv converter

In [77]:
import pandas as pd
import os

In [78]:
# this function read the files from the directory of the script
# return the files in the directory

def seekfiles():
    files = []
    for file in os.listdir("."):
        if file.endswith(".mpt"): # take all the files that ends with .mpt format and append the file into the list
            files.append(file)
    return files # return list of files with .mpt format 

In [79]:
# This function converts the given content of .mpt BT-LAB file into pandas dataset.
# parameter: filecontents --> the contents of the file from readlines()
# return: dataset --> return the dataset as a pandas dataframe

def convertToDataset(filecontents):
    newContent = []
    for lines in filecontents:
        newContent.append(lines.strip()) # delete the spacing \n from each lines
    headerLine = newContent[1][18:] # take the line number of the header in the .mpt file
    tableContent = newContent[int(headerLine)-1] # saved the column values of the data into a list
    header = tableContent.split('\t') # split the tableContent (headers list) from the tabular splitter.
    data = filecontents[int(headerLine):] # take the raw data starting from the header line + 1
    
    newData = [] # the data that is going to be used in making the pandas dataframe,
    for val in data:
        newData.append(val.rstrip().split('\t')) # append each raw data into newData with deleted tabular separator
    df = pd.DataFrame(newData, columns=header) # create a new pandas dataframe using newData list from before.
    dataset = df.apply(lambda x: x.str.replace(',','.')) # replace all comas in data into dots
    dataset = dataset.astype(float) # set all data from string into float
    return dataset 

In [80]:
# parameter: filecontents --> file content from .readlines
#            dataframe    --> pandas dataframe

def convertLoops(filecontents, dataframe):
    line = None
    
    # find the line where the loop number is located from the filecontent.
    for num, lines in enumerate(filecontents, 1):
        # find loop in the filecontent
        if 'loops' in lines:
            line = num
            
    # take the number of loops string from the .mpt file located in the line variable found above.        
    numLoops = filecontents[line-1][18:] 
    loops = filecontents[line:line+int(numLoops)] # a list of string of the number of loops
    
    # check whether there is a loop, if not then exit function.
    if len(loops) <= 1: 
        print("No Experiment Loops in the file.")
        return
    print('Number of loops: {}'.format(numLoops))
    
    loopStart = [] # -- X ROW -- starting point of new loop (x row) to loop end (y row) in lists
    loopEnd = [] # -- Y ROW -- end point of loop in lists
    
    for line in loops:
        loopStart.append(line.split()[5]) # take the starting point from the string list
    for line in loops:
        loopEnd.append(line.split()[7]) # take the end point from the string list
        
    dataLoops = []
    for i in range(len(loops)):
        # append from row x to row y specified by the loopStart and loopEnd variable above.
        dataLoops.append(dataframe.iloc[int(loopStart[i]):int(loopEnd[i])])
    return dataLoops

In [88]:
# output of seekfiles saved in a list

files = seekfiles()
files

['20220309_(Form02C)EIS_charging_10bis16V_step_01V+(1C+EIS)50cycle_16_GCPL6_C08.mpt',
 '20220309_(Form02C)EIS_charging_10bis16V_step_01V+(1C+EIS)50cycle_17_GEIS_C08.mpt']

In [82]:
# convert the files using the defined functions.

def toDF(files):
    datasets = []
    for file in files:
        print('Converting to .csv ... \nFilename: {} to .csv'.format(file))
        f = open(file,'r',encoding='ISO-8859-1')
        content = f.readlines()
        datasets.append(convertLoops(content, convertToDataset(content)))
    return datasets

In [83]:
def write_csv(dataset, files):
    for i, data in enumerate(dataset):
        print('converting {0} to .csv ...'.format(files[i]))
        for j, loops in enumerate(data):
            print('Converting loop {0} to .csv ...'.format(j))
            loops.to_csv('{file}_loop{loopNum}.csv'.format(file = files[i], loopNum=j), index=False)
        print('converted {0} to .csv'.format(files[i]))

In [89]:
# in action:

# put the files into the dfs list.

dfs = toDF(seekfiles())

Converting to .csv ... 
Filename: 20220309_(Form02C)EIS_charging_10bis16V_step_01V+(1C+EIS)50cycle_16_GCPL6_C08.mpt to .csv
Number of loops: 50

Converting to .csv ... 
Filename: 20220309_(Form02C)EIS_charging_10bis16V_step_01V+(1C+EIS)50cycle_17_GEIS_C08.mpt to .csv
Number of loops: 50



In [85]:
# To access a dataframe from the current folder: dfs[x][y] ----> x = database, y = loop index
# example below: 

dfs[0][0]

,mode,ox/red,error,control changes,Ns changes,counter inc.,Ns,time/s,dq/mA.h,(Q-Qo)/mA.h,...,Capacitance discharge/µF,<I>/mA,Q discharge/mA.h,Q charge/mA.h,Capacity/mA.h,Efficiency/%,control/V,control/mA,cycle number,Ecell/V
0,3.0,1.0,0.0,1.0,0.0,0.0,0.0,51497.493415,0.000000e+00,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,-3.073124
1,3.0,1.0,0.0,1.0,0.0,0.0,0.0,51498.493415,0.000000e+00,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,1.587269
2,3.0,1.0,0.0,1.0,0.0,0.0,0.0,51499.493415,0.000000e+00,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,1.587791
3,3.0,1.0,0.0,1.0,0.0,0.0,0.0,51500.493415,0.000000e+00,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,1.587912
4,3.0,1.0,0.0,1.0,0.0,0.0,0.0,51501.493415,0.000000e+00,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,1.587711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
960,2.0,0.0,0.0,0.0,0.0,0.0,2.0,63330.390637,-3.437103e-04,-0.136315,...,177090.875399,-0.010311,0.150451,0.0,0.150451,0.0,-0.000049,0.0,0.0,-0.002739
961,2.0,0.0,0.0,0.0,0.0,0.0,2.0,63400.710437,-1.969622e-04,-0.136512,...,177357.553116,-0.010083,0.150648,0.0,0.150648,0.0,-0.000049,0.0,0.0,-0.003018
962,2.0,0.0,0.0,0.0,0.0,0.0,2.0,63400.710637,-5.565228e-10,-0.136512,...,177345.938663,-0.010017,0.150648,0.0,0.150648,0.0,-0.000049,0.0,0.0,-0.002858
963,3.0,1.0,0.0,1.0,0.0,0.0,2.0,63400.759037,0.000000e+00,-0.136512,...,177345.938663,0.000000,0.150648,0.0,0.150648,0.0,0.000000,0.0,0.0,-3.072945


In [90]:
# convert each dataframe to csv, taking the files as a parameter for the file names.

write_csv(dfs, files)

converting 20220309_(Form02C)EIS_charging_10bis16V_step_01V+(1C+EIS)50cycle_16_GCPL6_C08.mpt to .csv ...
Converting loop 0 to .csv ...
Converting loop 1 to .csv ...
Converting loop 2 to .csv ...
Converting loop 3 to .csv ...
Converting loop 4 to .csv ...
Converting loop 5 to .csv ...
Converting loop 6 to .csv ...
Converting loop 7 to .csv ...
Converting loop 8 to .csv ...
Converting loop 9 to .csv ...
Converting loop 10 to .csv ...
Converting loop 11 to .csv ...
Converting loop 12 to .csv ...
Converting loop 13 to .csv ...
Converting loop 14 to .csv ...
Converting loop 15 to .csv ...
Converting loop 16 to .csv ...
Converting loop 17 to .csv ...
Converting loop 18 to .csv ...
Converting loop 19 to .csv ...
Converting loop 20 to .csv ...
Converting loop 21 to .csv ...
Converting loop 22 to .csv ...
Converting loop 23 to .csv ...
Converting loop 24 to .csv ...
Converting loop 25 to .csv ...
Converting loop 26 to .csv ...
Converting loop 27 to .csv ...
Converting loop 28 to .csv ...
Conver